In [1]:
import subprocess

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process import kernels
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from scipy import stats
import emcee
import numpy as np


%matplotlib inline

## Exercise 1 - Bayes Rule

Here, we're going to explore the mechanics of Bayesian analysis, and how prior choices can affect the posterior outcome. Let's revisit the coin example from the slides - we flipped 10 coins, and 7 came up heads. Our goal to make inference on the true probability of heads $\theta$, using Bayesian analysis. To do this, we need a likelihood and prior. 






In [2]:
coin_tosses = np.loadtxt('coin_tosses.txt')
num_heads = sum(coin_tosses)
N = len(coin_tosses)

**Likelihood**:
A common model for independent coin flips (or anything with a 0/1 outcome) is Binomial. So our likelihood is Binomial with size N = 10 and probability $\theta$:
\begin{align}
    y\sim \text{Binom}(N,\theta)
\end{align}

The mean of a Binomial random variable $Y$ with size $N$ (flips) and probability $\theta$ (probability of heads) is

\begin{align}
    \mathsf{E}(Y) = N\theta.
\end{align}

The variance of $Y$ is

\begin{align}
    \mathsf{V}(Y) = N\theta(1-\theta)
\end{align}

More information on Binomial distribution can found here: https://en.wikipedia.org/wiki/Binomial_distribution. 


**Prior**:
The most widely used prior for Binomial data and unknown probability is a Beta distribution. A Beta random variable can take value in (0,1).  

\begin{align}
    \theta\sim \text{Beta}(a, b)
\end{align}

The mean of a Beta random variable $\theta$ with first parameter $a$ and second paramter $b$ is

\begin{align}
    \mathsf{E}(\theta) = \frac{a}{a+b}
\end{align}

The variance of $\theta$ is

\begin{align}
    \mathsf{V}(\theta) = \frac{a + b}{(a+b)^2(a+b+1)}
\end{align}

More information on the Beta distribution can found here: https://en.wikipedia.org/wiki/Beta_distribution

Goal: Explore different priors for $\theta$ to see how the posterior of $\theta$ responds. 

#### Method 1: MCMC via python package emcee

Fill in missing pieces to calc_lnprior and calc_lnlike. These functions calculate the log of the prior and likelihood, respectively. For example, for the prior, you need to calculate the log of the pdf at theta for given parameters $a$ and $b$. Use the Wikipedia pages to find the pdfs.

    Hint 1: For the Beta(a,b) prior, you will need to calculate the Beta function of parameters a and b (see the Wikipedia page). The cell below imports betln, which calculates the natural log of the Beta function.

    Hint 2: For the Binomial likelihood, you'll need to calculate 
$$\left(\begin{matrix}N\\y\end{matrix}\right)$$
    
    This is the number of ways to get y heads in N flips. The function nCr(N,y) in the cell below calculates this.
    

In [3]:
from scipy.special import betaln
import math

def nCr(n,r):
    f = math.factorial
    return f(n) // f(r) // f(n-r)

print(nCr(4,2))

6


In [ ]:
#Calculate the log pdf of the prior of theta
def prior_ln_pdf(theta,a,b):
    #Ensure theta is within (0,1)
    if theta <=0 or theta >=1:
        return -np.inf
    
    ##Add return
    return ??

#Calculate the log pdf of the likelihood of y successes (heads) in N trials (flips)
def likelihood_ln_pdf(y, theta, N):

    ##Add function return
    return ??

def posterior_ln_pdf(theta,a,b,y, N):
    ln_pr = prior_ln_pdf(theta=theta,a=a,b=b)
    
    if not np.isfinite(ln_pr):
        return -np.inf
        
    ln_like = likelihood_ln_pdf(y=y, theta=theta,  N = N)
    return ln_pr + ln_like

    Hint 3: If done correctly:

    prior_ln_pdf(theta=0.4, a = 2,b = 3) = 0.547

    likelihood_ln_pdf(theta=0.5,y=num_heads,N=10) = -2.144

In [ ]:
##Check functions
print(np.round(prior_ln_pdf(theta=0.4, a = 2,b = 3),3))
print(np.round(likelihood_ln_pdf(theta=0.5,y=num_heads,N=10),3))

The code below sets up the sampler. $nsteps$ is the number of steps each walker will run, while $nburnin$ is the number of "burn-in" or "warmup" steps for each walker.  Our total number of samples will be $nwalkers\times (nsteps-nburnin)$ (recall we want 10,000 total samples). After running each walker for $nsteps$ and discarding the first $nburnin$, we reshape the output chain so each row is a draw from the posterior

In [ ]:
def get_posterior_draws(a,b, nsamples = 100, nburnin = 50, nwalkers = 200):
    ndim = 1
    sampler = emcee.EnsembleSampler(nwalkers, ndim, posterior_ln_pdf, args=(a, b, num_heads,N))
    p0 = np.random.rand(nwalkers,ndim)
    out_post = sampler.run_mcmc(p0,nsamples)
    samples = sampler.chain[:, nburnin:, :].reshape((-1, ndim))
    
    return samples

Use the samples to find posterior means and variances

In [ ]:
#Compare the prior density to the posterior samples
def plot_comparison_mcmc(prior_a, prior_b, nsamples = 100,nburnin = 50, nwalkers = 200):
    theta_grid = np.linspace(0.01,0.99,100)
    
    #Calculate the posterior density at the grid of theta values
    prior_pdf_vals = stats.beta(prior_a,prior_b).pdf(theta_grid)
    
    samples = get_posterior_draws(a = prior_a, b = prior_b,
                                  nsamples = nsamples, nburnin = nburnin, nwalkers = nwalkers)
    
    print('Prior mean is')
    print(prior_a/(prior_b+prior_b))
    print('Posterior mean is')
    print(np.round(samples.mean(),2))
    
    plt.plot(theta_grid,prior_pdf_vals,label = 'Prior Density' )
    plt.hist(samples,density = True,label = 'Posterior Samples',bins=20)
    plt.legend(loc='best', fontsize=12)
    pass

**Exercise**: run $plot\_comparison\_mcmc()$ with the following choices of a and b to compare the prior density to the histogram of posterior samples. Also report the prior mean versus the mean of the posterior samples

a = 1, b = 1

a = 1, b = 10

a = 10, b = 1

a = 30, b = 30

In [ ]:
plot_comparison_mcmc(??, ??)

#### Method 2: Direct computation of the posterior

We can find the analytic posterior in this case

First, recall the likelihood and prior

\begin{align}
	y\mid\theta&\sim \mathsf{Binom}(N,\theta)\\
	\theta&\sim \mathsf{Beta}(a,b)
\end{align}

Then, we apply Bayes Rule and do a little algebra

\begin{align}
	p(\theta\mid y)&\propto p(y\mid\theta)p(\theta)\\
	 &= \left[\begin{pmatrix}N \\ y\end{pmatrix}\theta^y(1-\theta)^{N-y}\right]\frac{\theta^{a-1}(1-\theta)^{b-1}}{B(a,b)}\\
	 &\propto \theta^{a+y-1}(1-\theta)^{b+N-y-1}
\end{align}
Note that when doing this algebra, we can disregard the constants $\begin{pmatrix}N \\ y\end{pmatrix}$ and $B(a,b)$ because they fall away in the proportionality.

At the end, we recognize the kernel of a $\mathsf{Beta}(a^*,b^*)$ distribution, where
\begin{align}
	a^* &= a + y\\
	b^* &= b + N - y
\end{align}
This is posterior distribution of $\theta$.

Note that this math was very simple only because of the choice of prior and likelihood; a Beta distribution is a so-called "conjugate prior" for a Binomial likelihood. For more comlicated models the posterior will not be available analytically, and we must resort to sampling methods. We only used sampling methods in this model for pedagogical purposes.

In [ ]:
def plot_comparison_analytic(prior_a,prior_b, posterior_a, posterior_b,
                            nsamples = 100,nburnin = 50, nwalkers = 200):

    theta_grid = np.linspace(0.01,0.99,100)
    
    #Calculate the prior pdf at the grid of theta values
    prior_pdf_vals = stats.beta(prior_a,prior_b).pdf(theta_grid)
    plt.plot(theta_grid, prior_pdf_vals, label = 'Prior')
    plt.xlabel('Theta')
    plt.ylabel('Pdf')
    
    #Compare to MCMC-generated results
    samples = get_posterior_draws(a = prior_a, b = prior_b, 
                                 nsamples = nsamples, nburnin = nburnin, nwalkers = nwalkers)
    plt.hist(samples,density = True,label = 'Posterior Samples', bins = 20)
    
    #Calculate the posterior pdf at the grid of theta values
    posterior_pdf_vals = stats.beta(posterior_a,posterior_b).pdf(theta_grid)
    plt.plot(theta_grid,posterior_pdf_vals, label = 'Analytic Posterior')
    
    
    print('Prior mean is')
    print(prior_a/(prior_b + prior_b))
    print('Analytic posterior mean is')
    print(np.round(posterior_a/(posterior_a + posterior_b),2))
    print('Posterior mean from MCMC is')
    print(np.round(samples.mean(),2))
    
    
    plt.legend(loc='best', fontsize=12)
    pass

**Exercise**: Find the parameters $a\_star$ and $b\_star$ for the posterior distribution of $\theta$ in our model. Compare the prior density to posterior density for the above choices of $a$ and $b$. Do the posterior densities align with the histogram of your posterior samples?

In [ ]:
a = #your choice
b = #your choice
N = len(coin_tosses)
y = sum(coin_tosses)
a_star = ??
b_star = ??

plot_comparison_analytic(prior_a = a, prior_b =  b,
                         posterior_a = a_star, posterior_b =  b_star)

## Exercise 2 - Latin Hypercube Design

In [ ]:
def generate_lhs(npoints, ndim, seed):
    """
    Generate a maximin Latin-hypercube sample (LHS) with the given number of
    points, dimensions, and random seed.

    """

    proc = subprocess.run(
        ['R', '--slave'],
        input="""
        library('lhs')
        set.seed({})
        write.table(maximinLHS({}, {}), col.names=FALSE, row.names=FALSE)
        """.format(seed, npoints, ndim).encode(),
        stdout=subprocess.PIPE,
        check=True
    )

    lhs = np.array(
        [l.split() for l in proc.stdout.splitlines()],
        dtype=float
    )


    return lhs

#### Create and plot design matrix below
##### 20 points, 2 dimensions, set the seed to 80

**Exercise**: Fill in the parameters for generate_lhs(), and for plt.scatter

In [ ]:
##Fill in the parameters to generate_lhs
design = generate_lhs(npoints = ??,
                      ndim = ??,
                      seed = ??)

In [ ]:
###Fill in parameters to plt.scatter
plt.scatter(x = ??,
           y = ??) 
plt.title('Latin Hypercube Design')
plt.xlabel('Input 1')
plt.ylabel('Input 2')
pass

## Exercise 3 - Toy GP Example

#### Part a - Mean and Variance Estimate

Run the two cells below

In [ ]:
def truth(x):
    return(3*x+np.cos(5*x))    
design = np.linspace(start =-1,stop=1,num=5)
model_data = truth(design)

In [ ]:
plt.scatter(x=design,y=model_data)
plt.title('Computer Model Output at Design Points')
plt.xlabel('Design')
plt.ylabel('Model Output')
pass

Run the cell below, training the GP

In [ ]:
ptp = 2
kernel = (
    1. * kernels.RBF(
        length_scale=ptp,
        length_scale_bounds=np.outer(ptp, (.1, 10))
    ) 
)
gp = GPR(kernel=kernel,
    #alpha=0,
    n_restarts_optimizer=0,
    copy_X_train=False).fit(design.reshape(-1,1), model_data)

**Exercise**: Create the vector of points X on which we will predict. X should be a (n x 1) numpy array (you choose n)

    Hint: We only want to predict within the bounds of our design

In [ ]:
X = ??
X = X.reshape(-1,1)

In [ ]:
#This returns the predictive mean and covariance at all the points in X
#mean is a (n,) numpy array, and cov is a (n,n) numpy array
mean, cov = gp.predict(return_cov=True,X=X)

**Exercise**: Fill in the lines for plt.plot, top_var, and bot_var

In [ ]:
#Set up the figure by first plotting the output at the design points
plt.scatter(x = design,y = model_data,color = 'black',label = 'Design Output')
plt.title('Computer Model Output at Design Points')
plt.xlabel('Input')
plt.ylabel('Model Output')


#Add the mean 95% uncertainty interval of the GP predictions at all the in-between points
##Fill in parameters to plt.plot
##Find expressions for top_var and bot_var
###### Hint: For a normal distribution, how many standard deviations away from the mean is the 97.5% quantile?
###### Hint: Use the diagonal of predictive covariance matrix`
plt.plot(X ,mean ,color= 'blue',label = 'GP Mean')
top_var = ??
bot_var = ??
plt.fill_between(X[:,0], bot_var, top_var, where=top_var >= bot_var, facecolor='lightgray', interpolate=True)

plt.plot(X,truth(X),color='black',label = 'Truth')
plt.legend(loc='best', fontsize=12)

pass

#### Part b - Random Draws

The prior exercises displayed the mean and variance of our function at all the points in X. But what about actual samples of the function itself? Here we visualize what a random draw of the function would look like.

**Exercse**: Fill in the lines for top_var, bot_var, and rand_draw

    Hint: For rand_draw, examine np.random.multivariate_normal()

In [ ]:
#Get the upper 95% quantile, and lower 95% quantile of the GP predictions at all the in-between points
##Fill in same values as previous exercise
top_var = ??
bot_var = ??

plt.scatter(design,model_data,color = 'black',label = 'Design Output')
plt.title('Computer Model Output at Design Points')
plt.xlabel('Input')
plt.ylabel('Model Output')
plt.fill_between(X[:,0], bot_var, top_var, where=top_var >= bot_var, facecolor='lightgray', interpolate=True)
plt.plot(X,truth(X),color='black',label = 'Truth')

ndraws = 10
colors = cm.rainbow(np.linspace(0, 1, ndraws))

#Get [ndraws] random draws from the predictive distribution of the GP at all of the in-between points 
#Use the predictive mean and covariance
#Hint: examine np.random.multivariate_normal()
rand_draw = ??
for i in range(ndraws):
    plt.plot(X,rand_draw[i,:],color = colors[i],linestyle = ":")

plt.legend(loc='best', fontsize=14)

pass


## Exercise 4 - Principal Component Analysis

The dataset below contains 20 developmental indices for 132 countries around the world. The country labels (rows) can be found in 'countries.txt,' and the index descriptions can be found in 'indices.txt.' Here we look to explore some of the practical uses of PCA for the purposes of Computer Emulation.

In [ ]:
developmental_indices = np.loadtxt('dev_indices.txt')

In [ ]:
scaler = StandardScaler(copy=False)
pca = PCA(copy=False, whiten=True, svd_solver='full')
Z = pca.fit_transform(scaler.fit_transform(developmental_indices))

**Exercise**: Plot the cummulative fraction of variance explained. How many PCs would you recommend using?

    Hint: Examine attributes of the object pca, as well as np.cumsum()

In [ ]:
F_r = ??

In [ ]:
plt.plot(range(len(F_r)),F_r,'-o')
ticks = np.arange(0, len(F_r), 1)
labels = ticks + 1
plt.xticks(ticks, labels)
plt.title('Fraction of Variance Explained')
plt.xlabel('Number of Components')
plt.ylabel('F_r')
pass

**Exercise**: Find the correlation between the first two principal component vectors

In [ ]:
corr = ??
corr

**Exercise**: Plot the second principal component against the first

In [ ]:
plt.scatter(??, ??)
plt.title('Principle Components')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
pass

## Exercise 5 - Bayes Rule with a "Toy Jet Quenching" example

### Things to practice

In this example, we use a very simple model of jet-quenching to practice all the techniques that we have learned above. Given a measurement: yexp +/- ystat +/- ysys, and a model M : parameters --> predictions,

1) Make a parameter design on which we run the computer model

2) Use Principal Component Analysis (PCA) to do dimension reduction

3) Build Gaussian Process emulators (GP) and train on the design

4) Construct proir, likelihood and posterior function

5) Use MCMC to marginalize the posterior distribution

6) Analyse the posterior distribution of the parameters

### A toy-model for jet quenching
The energy loss $\Delta E$ of a particle with energy $E$ follows a $\Gamma$-distribution,
\begin{equation}
P(\Delta E) = \Delta E ^ {\left(\mu/\sigma\right)^2-1} e^{-\mu\Delta E/\sigma^2}
\end{equation}
$\mu$ and $\sigma$ are the mean and std of the energy loss distribution, parametrized as
\begin{eqnarray}
\mu &=& A \sqrt{E},\\
\sigma &=& B\mu.
\end{eqnarray}
Therefore, this model has two parameters $A, B$. We assume that

1) The model is perfect (which is often not the case).

2) The true values are $A=1$ and $B=0.5$.

### Observables and measurements

The observable is an analog of $R_{AA}$. Given a reference spectrum,

\begin{eqnarray}
\frac{dN_0}{dp_T} = \frac{p_T}{\left(3^2 + p_T^2\right)^3}
\end{eqnarray}

And the $R_{AA}$ is calcualted as the ratio between the quenched and reference spectra,

\begin{eqnarray}
R_{AA} = \frac{dN_1/dp_T}{dN_0/dp_T} =  \frac{\int d\delta p_T P(\delta p_T) \frac{dN_0}{dp_T}(p_T+\delta p_T)}{dN_0/dp_T}
\end{eqnarray}

For constructing the measurements, we consider two types of uncertainty: uncorrelated statisitcal errors and correlated systematic errors. Although the correlation among uncertainties is not one of the major topics of this example, we will see at the very end that how different treatments of the correlation affect the posterior.

In [ ]:
from scipy.integrate import quad
from scipy.special import gammaln

# In this example, we use a very simple model of jet-quenching to practice all the
# all the techniques we learned above:
# Given a measurement: yexp +/- ystat +/- ysys,
#       and a model M : parameters --> predictions
# 1) Make a parameter design on which we run the computer model
# 2) Use Principal Component Analysis (PCA) to do dimension reduction
# 3) Build Gaussian Process emulators (GP) and train on the design
# 4) Construct proir, likelihood and posterior function
# 5) Use MCMC to marginalize the posterior distribution
# 6) Analyse the posterior distribution of the parameters


########### A simple model that calculates a "R_AA" ########################
# Baseline of particle production: dN0/dpT ~ pT/(3^2 + pT^2)^3
@np.vectorize
def dN0_dpT(pT):
    return pT/(3.**2 + pT**2)**3

# The spectrum after energy loss:
@np.vectorize
def dN1_dpT(pT, A, B):
    # dP: The probability of a particle with pT to loose delta_pT
    # Assume: 
    #     1) delta_pT follows a Gamma Distribution
    #     2) mean-pT-loss: <delta_pT> = A*sqrt(pT)
    #     3) pT-loss-fluctuation <delta_pT^2> - <delta_pT>^2 = B*<delta_pT>
    # The two parameters A and B are to be extracted from "data"
    def dP(delta_pT, pT, A, B):
        mean = A*pT**0.5
        std = B*mean
        alpha = mean**2/std**2
        beta = mean/std**2
        x = beta*delta_pT
        return np.exp( (alpha-1.)*np.log(x) - gammaln(alpha) - x ) * beta

    # The spectrum after energy loss is a convolution of dN0/dpT and dP
    # dN1/dpT = [integral] dN0/dpT(pT+Delta_pT) * dP(Delta_pT) * d[Delta_pT]
    def f(ln_1_delta_pT, pT, A, B):
        delta_pT = np.exp(ln_1_delta_pT) - 1.
        return dN0_dpT(pT+delta_pT) * dP(delta_pT, pT+delta_pT, A, B) * (1.+delta_pT)
    result, _, = quad(f, 0.0, np.log(1+5*pT), args=(pT, A, B))
    return result

# "Experimental data"
# Assume the model is perfect, and the truth values of A and B are 1.0 and 0.5
truth = [1., 0.5]
# The Measurement measure the truth Raa, subject to limited statistics and systematic bias
# yexp = y_true + ystat + ysys
# Default: 10% relative statistical uncertainty, ystat=0
@np.vectorize
def Measurement(StatLevel, SysLevel):
    np.random.seed(10)
    pTbin = np.array([1,2,3,4,5,6,10,15,20,30,40,60,100])
    pT = (pTbin[1:] + pTbin[:-1])/2.
    be = (pTbin[1:] - pTbin[:-1])/2.
    y_truth = dN1_dpT(pT, truth[0], truth[1])/dN0_dpT(pT)
    results = y_truth * np.random.normal(1.0, StatLevel, pT.shape[0]) \
              * np.random.normal(1.0, SysLevel)
    return pT, be, y_truth, results, StatLevel*results, SysLevel*y_truth

# The model without known the true value of A and B
# With what we have learned, the probablity distribution of A, B
# will be inferred from data using this model (a perfect model).
@np.vectorize
def Model(pT, A, B):
    return dN1_dpT(pT, A, B)/dN0_dpT(pT)


In [ ]:
################Step 1: Get the Measurement########################
#  Choose the magnitudes of Stat and Sys error, (recommanded 10%)
#  1) plot the true Raa v.s. pT
#  2) plot the experimental data with stat and sys error
#
###################################################################

pT, pTbin, ytruth, yexp, ystat, ysys =\
    Measurement(StatLevel=.05, SysLevel=.1)
    
# plot True Raa here
plt.plot(??, ??, 'k.', label='Truth') 

# plot Raa measurement with stat errorbars
plt.errorbar(x=??, xerr=??, y=??, yerr=??,
             fmt='rD', label=r"$y \pm$ stat") 

# plot sys errorband, y1:lowerbounds, y2: higherbounds
plt.fill_between(x=pT, y1=yexp-??, y2=yexp+??,
                 color='r', alpha=.3, label=r"$\pm$ sys")

plt.ylim(0,1)
plt.semilogx()
plt.xlabel(r'$p_T$ [GeV]', fontsize=15)
plt.ylabel(r'$R_{AA}$', fontsize=15)
pass

In [ ]:
################Step 2: ###################################
# Make a design over the parameter space (A, B)
# 1) What is a reasonable range of the prior? 
# ( A and B cannot be too close to 0 due to numerics )
# 2) Generate the design and rescale it to the desired range
#    Hint: linear rescale x from (0,1) to y from (a,b):
#           y = (1-x)*a + x*b
# 3) Run model() on each design points
#    The design matrix model_data should have a shape: 
#           N_design x N_pT
rangeA = [??, ??]
rangeB = [??, ??]
ranges = np.array([rangeA, rangeB])
hypercube = generate_lhs(npoints=??, ndim=??, seed=80)
design = ??????????????? # rescale the hypercube to desired range
# Run model on design points
model_data = ???????????????????


# plot design points
plt.subplot(1,2,1)
plt.scatter(design.T[0], design.T[1])
plt.xlabel("$A$", fontsize=15)
plt.ylabel("$B$", fontsize=15)
# plot all calulations
plt.subplot(1,2,2)
for y in model_data:
    plt.plot(pT, y, 'r-', alpha=0.3)
    plt.ylim(0,1)
plt.semilogx()
plt.xlabel(r'$p_T$ [GeV]', fontsize=15)
plt.ylabel(r'$R_{AA}$', fontsize=15)
plt.tight_layout(True)

In [ ]:
###### Step 3: apply PCA ######################################
# We don't need a separate GP for each pT point
# 1) Try keeping different number of principal components (npc).
#    How many pc(s) do you think is enought for this exercises?
# 2) Take a look at the feature of each pc (figure 3).
#    What each pc does in terms of decomposing the data?
# 3) Look at the coorelation between PC1 and PC2 (figure 2),
#    are they completely uncorrelated? Combine with your
#    observations from (figure 3), can you explain what
#    causes the correlation?
################################################################
npc = ??
scaler = StandardScaler(copy=True)
pca = PCA(copy=True, whiten=True, svd_solver='full')

# Keep only the first `npc` principal components
Z = pca.fit_transform(scaler.fit_transform(model_data))[:,:npc]

# The transformation matrix from PC to Physical space
Trans_Matrix = (  pca.components_
                * np.sqrt(pca.explained_variance_[:, np.newaxis])
                * scaler.scale_)

# Estimate the covariance of the negelected PCs
Residual_Cov = np.dot(Trans_Matrix[npc:].T, Trans_Matrix[npc:])

# plotting
plt.figure(figsize=(15,4))
F_r = np.cumsum(pca.explained_variance_ratio_)
plt.subplot(1,3,1)
plt.plot(range(len(F_r)), F_r, '-o')
ticks = np.arange(0, len(F_r), 1)
labels = ticks + 1
plt.xticks(ticks, labels)
plt.title('Fraction of Variance Explained')
plt.xlabel('Number of Components')
plt.ylabel('F_r')

plt.subplot(1,3,2)
plt.scatter(Z[:, 0], Z[:, 1])
plt.xlabel('$Z_0$', fontsize=15)
plt.ylabel('$Z_1$', fontsize=15)

plt.subplot(1,3,3)
for i, (comp, color) in \
        enumerate(zip(pca.components_, 'rgb')):
    plt.plot(pT, comp, color=color, label='PC {}'.format(i+1))
plt.xlabel('$p_T$', fontsize=15)
plt.ylabel('$Features$', fontsize=15)
plt.tight_layout(True)

In [ ]:
######## Step 4-1: Building Emulators #############################
# Using an Exp-Squared kernel + a white kernel (accounting 
# for numerical error of model calculations)
# 1) Put in initial length scales for param A and B
# 2) Put in reasonable lenght scales bounds for optimization
# 3) Fit separate emulatior to each principal component.
#    Take a look at the optimized hyper-parameters.
#    What do they mean?
kernel = (
    1. * kernels.RBF(
        length_scale=[??, ??],
        length_scale_bounds=[(??, ??), (??, ??)]
    )  
    + kernels.WhiteKernel(.1)
)

# Build and train each GP
gps = [ GPR(kernel=kernel, n_restarts_optimizer=10) 
        for i in range(npc) ]
for i, gp in enumerate(gps):
    gp.fit(??, ??)
    print('RBF: ', gp.kernel_.get_params()['k1'])
    print('White: ', gp.kernel_.get_params()['k2'])

In [ ]:
### Step 4-2: Validating the emulators #######################
# It is important to validate the performance of emulators to
# make sure they behave as expected.
# 1) Pick 6 random combinations of A and B. Compare the
#    emulators prediction and the model calculations.
#    Do they agree? 
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True)

for (a, b) in [(.4, .22), (??, ??), (??, ??), 
               (??, ??),  (??, ??), (??, ??)]:
    # GP prediction
    z = np.array([gp.predict([(a, b)])[0] for gp in gps])
    pred = np.dot(z, Trans_Matrix[:z.shape[-1]])
    pred += scaler.mean_
    
    # model calcuatlion
    calc = Model(pT, a, b)
    
    ax1.plot(pT, calc, 'ro', alpha=0.7)
    ax1.plot(pT, pred, 'b--', alpha=0.7)
    ax2.plot(pT, (pred-calc)/calc, 'b--', alpha=0.7)

ax1.semilogx()
ax1.set_xlabel(r'$p_T$ [GeV]', fontsize=15)
ax1.set_ylabel(r'$R_{AA}$', fontsize=15)
ax2.set_ylim(-.2, .2)
ax2.set_xlabel(r'$p_T$ [GeV]', fontsize=15)
ax2.set_ylabel('relative error', fontsize=15)

plt.tight_layout(True)

In [ ]:
##### Helper functions for this block ###################
from scipy.linalg import lapack
# calculate the log of Gaussian density with 
# residual dy = y-mu and covariance matrix cov.
# - 1/2 * dy^T * cov^[-1] * dy - 1/2*ln(|cov|)
def lnLL(dy, cov):
    L, info = lapack.dpotrf(cov, clean=False)
    alpha, info = lapack.dpotrs(L, dy)
    return -.5*np.dot(dy, alpha)-np.log(L.diagonal()).sum()

# Transform a covariance matrix from the PC space 
# back to the physical space
def transform_cov(std):
    cov = np.matmul(Trans_Matrix[:npc].T*std**2, 
                    Trans_Matrix[:npc])\
        + Residual_Cov 
    return cov

####### Step 5: Construct the posterior #################
# Remember that from Bayes' Theorem:
#      Posterior  = prior * likelihood
# and:
#      ln(Posterior) = ln(prior) + ln(likelihood)
# and:
#      theta = [A, B]
# 1) Complete the returns of the prior "prior_ln_pdf(theta)"
# 2) The sys-error is correlated, while the stat one is not
#    We provide two types of covariance matrixx
#        2.1) cov_exp1 treats sys-error as uncorrelated
#        2.2) cov_exp2 treats sys-error as correlated
#    Start with 2.1) and later try 2.2) to see the effects
#    on the posterior distribution of A and B.
# 3) Complete the likelihood_ln_pdf(theta) function
def prior_ln_pdf(theta):
    if (theta<ranges[:,0]).any() or (theta>ranges[:,1]).any():
        return -np.inf
    else:
        return 0.

# Pick your experimental covariance matrix
Assume_SysError_Corr = ?? # try True or False
cov_exp = np.diag(ystat**2) + np.outer(ysys, ysys) \
          if Assume_SysError_Corr else \
          np.diag(ystat**2) + np.diag(ysys**2)

def likelihood_ln_pdf(theta):
    z, stdz = np.array([gp.predict([theta], return_std=True) for gp in gps]).T[0]
    pred = np.dot(z, Trans_Matrix[:z.shape[-1]])
    pred += scaler.mean_
    cov_emulator = transform_cov(std=stdz)
    dy = ??
    cov = ??
    return lnLL(dy, cov)

# Finally ln(Posterior) = ln(prior) + ln(likelihood)
def posterior_ln_pdf(theta):
    ln_pr = prior_ln_pdf(theta)
    ln_like = likelihood_ln_pdf(theta) 
    return ln_pr + ln_like


In [ ]:
######### Step 6: Run MCMC ###########################
# Fill 1) the number of samples 2) burnin steps
# 3) dimsional of the problem 4) number of mcmc walkers
# Hint: this may take a while, so start with smaller numbers
nsteps = ??
nburnin = ??
ndim = ??
nwalkers = 20
sampler = emcee.EnsembleSampler(nwalkers, ndim, 
                                posterior_ln_pdf)
p0 = np.random.rand(nwalkers, ndim)
p0 = (1.-p0)*ranges[:, 0] +  p0*ranges[:, 1]
out_post = sampler.run_mcmc(p0, nsteps)
samples = sampler.chain[:, nburnin:, :].reshape((-1, ndim))

In [ ]:
##### Step 7: Analyze the posterior distribution ########
# 1) Run this block and plot the posterior distribution
# 2) Does the posterior fairly estimates the true values (red)?
# 3) How does the posterior change it we take into account the
#    correlation among the sys-error?
figure, axes = plt.subplots(figsize=(5,5), 
                            ncols=ndim, nrows=ndim)
names = [r"$A$", r"$B$"]
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        if i==j:
            ax.hist(samples[:,i], bins=40,
                    range=ranges[i], histtype='step', 
                    density=True)
            ax.set_xlabel(names[i])
            ax.axvline(x=truth[i], color='r', linewidth=1)
            ax.set_xlim(*ranges[j])
        if i>j:
            ax.hist2d(samples[:, j], samples[:, i], 
                      bins=40, range=[ranges[j], ranges[i]], 
                      cmap='Blues')
            ax.set_xlabel(names[j])
            ax.set_ylabel(names[i])
            ax.axvline(x=truth[j], color='r', linewidth=1)
            ax.axhline(y=truth[i], color='r', linewidth=1)
            ax.plot(truth[j], truth[i], 'ro')
            ax.set_xlim(*ranges[j])
            ax.set_ylim(*ranges[i])
        if i<j:
            ax.axis('off')
plt.tight_layout(True)


In [ ]:
# predicting observables
param_samples = samples[ np.random.choice(range(len(samples)),50), :]
z  = np.array([gp.predict(param_samples) for gp in gps]).T
pred = np.dot(z, Trans_Matrix[:z.shape[-1]])
pred += scaler.mean_
for i, y in enumerate(pred):
    plt.plot(pT, y, 'b-', alpha=0.15, label="Posterior" if i==0 else '')
plt.errorbar(pT, yexp, yerr=ystat, xerr=pTbin, fmt='ro', label="Measurements")
for xl, xr, yl, yh in zip(pT-pTbin, pT+pTbin, yexp-ysys, yexp+ysys):
    plt.fill_between([xl,xr],[yl, yl], [yh, yh], facecolor='None', edgecolor='r')
plt.ylim(0,1)
plt.semilogx()
plt.xlabel(r'$p_T$ [GeV]', fontsize=15)
plt.ylabel(r'$R_{AA}$', fontsize=15)
plt.legend()
pass